In [1]:
import numpy as np
import pandas as pd


In [2]:
import csv
import numpy as np

cnt = 1
csv_file_path = 'crop_production.csv'
states = {}
districts = {}
season_to_number = {}
crops = {}
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in  csv_reader:
        cnt = cnt+1
        if cnt == 1 : continue
        states[row[0].strip()] = 1
        districts[row[1].strip()] = 1
        season_to_number[row[3].strip()] = 1
        crops[row[4].strip()] = 1
        
cnt = 1
for val in states.keys():
    states[val] = cnt
    cnt = cnt+1
cnt = 1
for val in districts.keys():
    districts[val] = cnt
    cnt = cnt+1
cnt = 1

for val in season_to_number.keys():
    season_to_number[val] = cnt
    cnt = cnt+1
cnt = 1
for val in crops.keys():
    crops[val] = cnt
    cnt = cnt+1




In [3]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split


csv_file_path = 'crop_production.csv'

y = []
data = []
cnt = 0
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        cnt = cnt+1
        if cnt == 1 : continue
        try:
            y.append(float(row[6]))
        except:
            continue
        temp_row = row
        if len(temp_row[1]) ==0 : continue
        if len(temp_row[2]) ==0 : continue
        if len(temp_row[3]) ==0 : continue
        if len(temp_row[4]) ==0 : continue
        if len(temp_row[5]) ==0 : continue
        temp_row[1] = districts[temp_row[1].strip()]
        temp_row[2] = int(temp_row[2])
        temp_row[3] = season_to_number[temp_row[3].strip()]
        temp_row[4] = crops[temp_row[4].strip()]
        temp_row[5] = float(temp_row[5])
        
        temp1 = row[1:2]
        temp2 = row[3:6]
        data.append(temp1+temp2)
       
y = np.array(y)

data = np.array(data)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(data)



X_train, X_test, y_train, y_test = train_test_split(X_train_normalized, y, test_size = .25, random_state = 18)


In [4]:
# n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8 

In [35]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

param_grid_gb = {
    "eta" : "1e-07",
    "n_estimators" : [5000],
    "min_child_weight" : [20,40,60,80,100],
    "max_depth" : [2,4,6,8,10]
}

model = XGBRegressor()
mse_grid = GridSearchCV(estimator=model, param_grid=param_grid_gb, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)

mse_grid.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=20)

print("Best parameters:", mse_grid.best_params_)


# Procedure: First tune min_child_weight and max_depth
# Tune gamma
# Tune subsample and colsample
# Tune regularization (alpha paramater)
# Tune the estimators as required

# Previous run (27m 35.4s) Best parameter:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1}
# Best parameter:  {'colsample_bytree': 1e-05, 'gamma': 0, 'learning_rate': 1e-07, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 100, 'reg_alpha': 1e-07, 'subsample': 1}


Fitting 5 folds for each of 4 candidates, totalling 20 fits
[0]	validation_0-rmse:15071978.96218
[1]	validation_0-rmse:12673209.18592
[2]	validation_0-rmse:11090021.03559
[3]	validation_0-rmse:10154383.28141
[4]	validation_0-rmse:9604389.80627
[5]	validation_0-rmse:8682819.77421
[6]	validation_0-rmse:8150145.76322
[7]	validation_0-rmse:7756393.25851
[8]	validation_0-rmse:7440021.25459
[9]	validation_0-rmse:7144494.70875
[10]	validation_0-rmse:6907726.19249
[11]	validation_0-rmse:6739506.86044
[12]	validation_0-rmse:6609087.54904
[13]	validation_0-rmse:6581999.59900
[14]	validation_0-rmse:6497850.21727
[15]	validation_0-rmse:6463075.74587
[16]	validation_0-rmse:6462314.65611
[17]	validation_0-rmse:6462309.97869
[18]	validation_0-rmse:6463623.14272
[19]	validation_0-rmse:6387395.72424
[20]	validation_0-rmse:6384384.18321
[21]	validation_0-rmse:6378557.41281
[22]	validation_0-rmse:6370009.48721
[23]	validation_0-rmse:6377691.84647
[24]	validation_0-rmse:6446229.05137
[25]	validation_0-rms

In [6]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV

# param_random_gb = {
#               'learning_rate': np.arange(0.05,0.15, 0.05),
#               # 'n_estimators' : np.arange(200,1000,100)
#             #   'subsample' : np.arange(0.9,1, 0.05),
#             #   "colsample_bytree" : np.arange(0.9,1,0.05)
#               }

# # Regressor Instantiation
# gb = XGBRegressor()

# # Here we specified a total of 25 iterations
# mse_random = RandomizedSearchCV(estimator = gb, param_distributions= param_random_gb, n_iter = 25, 
#                                 scoring = 'neg_mean_squared_error', cv = 4, verbose = 1)

# mse_random.fit(X_train_normalized1,y_train1)

# print("Best parameter: ", mse_random.best_params_)

In [74]:
# Best parameter:  {'colsample_bytree': 1, 'gamma': 1e-05, 'learning_rate': 1e-07, 'max_depth': 4, 'min_child_weight': 12, 'n_estimators': 100, 'reg_alpha': 1e-08, 'subsample': 0.9}


model = XGBRegressor(n_estimators=1000,eta = 1e-03)
# eval_set = [(X_test,y_test)]
model.fit   (X_train,y_train,eval_set=[(X_train,y_train),(X_test,y_test)],early_stopping_rounds=20,)

# print("Best parameter: ", mse_grid.best_params_)

[0]	validation_0-rmse:16462948.70985	validation_1-rmse:18744794.03756
[1]	validation_0-rmse:16449650.39036	validation_1-rmse:18730134.09697


c:\Python311\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[2]	validation_0-rmse:16436366.79819	validation_1-rmse:18715489.46542
[3]	validation_0-rmse:16423098.46872	validation_1-rmse:18700861.23290
[4]	validation_0-rmse:16409844.62392	validation_1-rmse:18686248.14085
[5]	validation_0-rmse:16396606.01619	validation_1-rmse:18671650.93750
[6]	validation_0-rmse:16383381.92925	validation_1-rmse:18657079.59017
[7]	validation_0-rmse:16370172.96500	validation_1-rmse:18642514.08428
[8]	validation_0-rmse:16356978.45653	validation_1-rmse:18627974.27303
[9]	validation_0-rmse:16343799.17274	validation_1-rmse:18613450.25334
[10]	validation_0-rmse:16330634.14898	validation_1-rmse:18598931.11905
[11]	validation_0-rmse:16317484.10288	validation_1-rmse:18584438.05553
[12]	validation_0-rmse:16304348.99159	validation_1-rmse:18569950.65028
[13]	validation_0-rmse:16291228.12689	validation_1-rmse:18555488.34760
[14]	validation_0-rmse:16278122.42815	validation_1-rmse:18541031.98933
[15]	validation_0-rmse:16265031.16184	validation_1-rmse:18526601.15530
[16]	validatio

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.001, eval_metric=None,
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, predictor=None, ...)

In [63]:
# training_error = 0
from sklearn.metrics import mean_squared_error
print("Training error:")
print(mean_squared_error(y_train,model.predict(X_train)))
print("Test error:")
print(mean_squared_error(y_test,model.predict(X_test)))
# Previously
# Training error:
# 2041076666.4211206
# Test error:
# 71631908865818.11
# Overfit
# Now
# Training error:
# 444956956552237.3
# Test error:
# 61499447426576.72
# Ok

Training error:
130332238542983.11
Test error:
172212765604488.94


In [83]:
# This will be used for user inputs
# State_Name = states['Andaman and Nicobar Islands']
District_Name = districts['NICOBARS']
# Crop_Year = 2000
Season = season_to_number['Whole Year']
Crop = crops['Coconut']
Area = 18190.00
# print(State_Name,District_Name,Crop_Year,Season,Crop,Area)
X_predict = np.zeros((1,4))
X_predict[0] = np.array([District_Name,Season,Crop,Area])
X_pred_normalized = scaler.transform(X_predict)

Production = model.predict(X_pred_normalized)
print(X_pred_normalized)
print(Production)

[[-1.65923103 -0.30775671 -1.07061753  0.11842118]]
[66383200.]


In [11]:
import pickle
filename = "finalized_model.sav"
# pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [67]:
import pickle
filename = "finalized_model2.sav"
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
#  Does not perform well on small aread like 1 10 100
# load the model from disk
# model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)